In [4]:
import numpy as np
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180/np.pi)
    
    if angle > 180:
        angle = 360 - angle
    
    return round(angle,1)

In [6]:
import mediapipe as mp
import cv2
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

stage = ''
counter = 0


cap = cv2.VideoCapture(0)
cv2.namedWindow("Mediapipe", cv2.WINDOW_AUTOSIZE)

with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if cv2.waitKey(10) == ord("q"):
            break
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        results = pose.process(image)
        
        try:
            landmarks = results.pose_landmarks.landmark
            # print(landmarks)
            showlder = [landmarks[12].x, landmarks[12].y]
            elbow = [landmarks[14].x, landmarks[14].y]
            wrist = [landmarks[16].x, landmarks[16].y]
            
            angle = calculate_angle(showlder, elbow, wrist) # calculate the angle
            # print(angle)
            cv2.putText(frame, str(angle), tuple(np.multiply(elbow, [640, 480]).astype(int)),
                                           cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            
            if angle > 150:
                stage = "down"
            elif angle < 45 and stage == "down":
                stage = "up"
                counter += 1
                print(counter)        
            
            
            
        except Exception as e:
            print(e)
        
        
        cv2.putText(frame, str(counter), (10,10), (100, 100, 100), -1)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color = (245, 117, 234), thickness = 2, circle_radius = 2),
                                 mp_drawing.DrawingSpec(color = (23, 23, 100), thickness= 2, circle_radius = 4))
        mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp.holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color = (245, 117, 234), thickness = 2, circle_radius = 2),
                                 mp_drawing.DrawingSpec(color = (23, 23, 100), thickness= 2, circle_radius = 4))
        mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color = (245, 117, 234), thickness = 2, circle_radius = 2),
                                 mp_drawing.DrawingSpec(color = (23, 23, 100), thickness= 2, circle_radius = 4))
        mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                 mp_drawing.DrawingSpec(color = (245, 117, 234), thickness = 2, circle_radius = 2),
                                 mp_drawing.DrawingSpec(color = (23, 23, 100), thickness= 2, circle_radius = 4))
        
        cv2.imshow("Mediapipe", frame)

    cap.release()
    cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'mediapipe'

In [3]:
!pip install mediapipe opencv-python pandas scikit-learn

ERROR: Could not find a version that satisfies the requirement mediapipe (from versions: none)
ERROR: No matching distribution found for mediapipe


In [10]:
#import mediapipe as mp
import cv2
import csv
import os
import numpy as np

In [9]:
num_coords = len(results.face_landmarks.landmark) + len(resilts.pose_landmarks.landmark)
num_coords

NameError: name 'results' is not defined

In [ ]:
landmarks = ["class"]
for val in range(1, num_coords + 1):
    landmarks += [f'x{val}', f'y{val}', f'z{val}', f'v{val}']

In [ ]:
with open('coords.csv', mode = 'w', newline = '') as f:   # a -- append; w -- write; r -- read
    csv_writer = csv.writer(f, delimiter = ',', quotechar = '""', quoting = csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)   # writerow -- write in row

In [ ]:
try:
    pose = results.pose_landmarks.landmark
    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]f for landmark in pose]).flatten())
    
    face = results.face_landmarks.landmark
    face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]f for landmark in pose]).flatten())
    
    row = pose_row + face_row
    row.insert(0, class_name)
    with open('coords.csv', mode = 'a', newline = '') as f:   # a -- append; w -- write; r -- read
        csv_writer = csv.writer(f, delimiter = ',', quotechar = '""', quoting = csv.QUOTE_MINIMAL)
        csv_writer.writerow(row)   # writerow -- write in row
        
except:
    pass
        
        
            
        
        

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('coords.csv')


# df.head()

# df.tail()

In [ ]:
X = df.drop('class', axis = 1)
y = df['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1234)

In [ ]:
print(len(X_train))

In [2]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
pipelines = {
    'lr' : make_pipeline(StandardScaler(), LogisticRegression())
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train.values, y_train.values)
    fit_models[algo] = model

In [ ]:
fit_models

In [ ]:
fit_models['lr'].predit(X_test.values)

In [ ]:
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test.values)
    print(algo, accuracy_score(y_test.values, yhat))

In [ ]:
import pickle
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)                     # save trained model

In [ ]:
import pickle
with open('body_language.pkl', 'wb') as f:               # load saved model
    model = pickle.load(f)